In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import json
import ta
import matplotlib.pyplot as pl

In [1]:
all_stocks = ['NVR', 'BKNG', 'CMG', 'AZO' ]

In [11]:
start_date = '2013-01-01'
end_date = '2023-03-31'

# Download the data
df = yf.download(all_stocks, start=start_date, end=end_date)['Adj Close']
df = df.reset_index()
# Save the data to a CSV file
df.to_csv('stocks_data.csv')

# Display the 1st 5 records
df.head(5)

[*********************100%%**********************]  4 of 4 completed


Ticker,Date,AZO,BKNG,CMG,NVR
0,2013-01-02,356.880005,635.308105,6.0212,930.130005
1,2013-01-03,356.140015,645.588074,6.0190,949.000000
2,2013-01-04,360.850006,645.269592,6.0036,954.489990
3,2013-01-07,356.170013,654.196167,5.9918,966.890015
4,2013-01-08,348.250000,654.235901,5.9552,974.500000


Ticker,Date,AZO,BKNG,CMG,NVR
0,2013-01-02,356.880005,635.308105,6.021200,930.130005
1,2013-01-03,356.140015,645.588074,6.019000,949.000000
2,2013-01-04,360.850006,645.269592,6.003600,954.489990
3,2013-01-07,356.170013,654.196167,5.991800,966.890015
4,2013-01-08,348.250000,654.235901,5.955200,974.500000
...,...,...,...,...,...
2574,2023-03-24,2329.399902,2487.225098,32.485001,5413.000000
2575,2023-03-27,2368.550049,2496.161621,32.956402,5375.450195
2576,2023-03-28,2405.219971,2532.793701,33.057999,5413.560059
2577,2023-03-29,2395.370117,2559.314453,33.134399,5433.990234


In [7]:
# plot the 4 stocks together 
df.droplevel('Ticker')

pd.melt(df, id_vars=Date, value_vars=None, var_name=None, value_name='value', col_level=None, ignore_index=True)


KeyError: 'Requested level (Ticker) does not match index name (Date)'